In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import xgboost
import sys
 
sys.path.append('C:\\Users\\Nikhil\\Developer\\Data_Science_Projects')
from common_ds_modules import missing_values, data_manipulation, modeling, eda
import os
import pandas as pd
import numpy as np
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
import seaborn as sns
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import Lasso
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
import pickle
from sklearn.linear_model import Lasso, Ridge, ElasticNet
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import StackingClassifier, GradientBoostingClassifier

from sklearn.compose import make_column_transformer 
from sklearn.pipeline import make_pipeline

from sklearn.preprocessing import FunctionTransformer
log_transformer = FunctionTransformer(np.log1p)

from sklearn.compose import make_column_selector as selector
from scipy.stats import skew
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import VotingClassifier

In [3]:
MAX_MODELS = 50

In [22]:
def get_columns(df, query=''):
    return [c for c in df.columns if query.lower() in c.lower()]

In [5]:
def get_pipeline_cross_val_score(preprocessor, model, param_grid, x_train, y_train, scoring_function,max_models=None):
    print(f'Max models: {max_models}')
    if model.__name__ == 'Lasso' or model.__name__ == 'Ridge' or model.__name__ == 'ElasticNet':
        print(f'Scaling Pipe: {model.__name__}')
        pipe = make_pipeline(preprocessor, StandardScaler(), model())
    else:
        pipe = make_pipeline(preprocessor, model())
    
    if max_models is not None:
        random_search = RandomizedSearchCV(pipe, param_grid, scoring=scoring_function,
                                   n_iter=max_models, verbose=0)
    else:
        random_search = GridSearchCV(pipe, param_grid, scoring=scoring_function,
                                   n_iter=max_models, verbose=0)

    random_search = random_search.fit(x_train, y_train)
    best_params = { k.split('__')[1]:v for (k,v) in zip(list(random_search.best_params_.keys()), list(random_search.best_params_.values()))} 
    if model.__name__ == 'Lasso' or model.__name__ == 'Ridge' or model.__name__ == 'ElasticNet':
        print(f'Scaling Final Pipe: {model.__name__}')
        final_pipe = final_pipe = make_pipeline(preprocessor, StandardScaler(), model(**best_params))
    else:
        final_pipe = make_pipeline(preprocessor, model(**best_params))
    
    score = cross_val_score(final_pipe, x_train, y_train, cv=5, scoring=scoring_function).mean()
    
    return final_pipe, score, best_params

In [6]:
xgb_param_grid = {
    'xgbclassifier__n_estimators': [110, 120],
    'xgbclassifier__max_depth':[5,6],
    'xgbclassifier__max_leaves':[40, 30, 50],
    'xgbclassifier__reg_alpha': [i/100 for i in range(1,3)],
    'xgbclassifier__reg_lambda': [i/100 for i in range(1,3)],
    'xgbclassifier__colsample_bytree': [i/10 for i in range(2, 5)],
    'xgbclassifier__learning_rate': [i/100 for i in range(3,6)],
    'xgbclassifier__gamma': [i/100 for i in range(1, 3)]
}

lasso_param_grid = {
    'lasso__alpha':[0.01, 0.05, 0.1, 0.5, 1, 5, 10, 50, 75, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]
}

ridge_param_grid = {
    'ridge__alpha':[0.01, 0.05, 0.1, 0.5, 1, 5, 10, 50, 75, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]
}

elastic_net_param_grid = {
    'elasticnet__alpha': [0.01, 0.05, 0.1, 0.5, 1, 5, 10, 50, 75, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000],
    'elasticnet__l1_ratio':[0.01, 0.05, 0.1, 0.5, 1]
}

knn_param_grid = {
    'kneighborsclassifier__weights' : ['uniform'],
    'kneighborsclassifier__algorithm' : ['auto'],
    'kneighborsclassifier__n_neighbors' : [5,10,15, 20, 25, 30],
    'kneighborsclassifier__leaf_size': [10],
    'kneighborsclassifier__p': [1],
}

In [7]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [8]:
label = 'Survived'
y_train = train_df[label]

id_column = 'PassengerId'
pred_col = f'Predicted_{label}'

In [9]:
numerical_variables, categorical_variables = data_manipulation.get_numerical_categorical_variables(train_df,\
                                                                                            test_df, id_column)                                                                                                  

In [10]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer

In [11]:
def plot_variable_dist(df, variables):
    for c in variables:
        print(f'Variable: {c}')
        display(df[c].describe())
        plt.hist(df[c])
        plt.title(f'Distribution for {c}')
        plt.show()
        
def plot_discrete_variable_dist(df, variables):
    for c in variables:
        print(f'Variable: {c}')
        display(df[c].value_counts(normalize=True))
        plt.hist(df[c])
        plt.title(f'Distribution for {c}')
        plt.show()
        
def plot_cat_variable_dist(df, variables):
    for c in variables:
        print(f'Variable: {c}')
        display(df[c].value_counts(normalize=True))
        plt.bar(df[c].value_counts().index, df[c].value_counts())
        plt.title(f'Distribution for {c}')
        plt.show()

In [12]:
def predict(x_train, y_train, x_test, model, id_col_name, id_col, pred_col, label, filename):
    model = model.fit(x_train, y_train)
    test_pred = model.predict(x_test)
    
    x_test[pred_col] = test_pred
    x_test[id_col_name] = id_col
    
    pred = x_test[[id_col_name, pred_col]]
    columns = dict()
    columns[pred_col] = label
    pred = pred.rename(columns=columns)
    pred.to_csv(filename, index=False)

In [13]:
missing_value_df = missing_values.get_variable_missing_values(train_df).reset_index(drop=True).sort_values('percent_missing', ascending=False)
missing_value_df

,column_name,percent_missing
10,Cabin,77.104377
5,Age,19.865320
11,Embarked,0.224467
0,PassengerId,0.000000
1,Survived,0.000000
2,Pclass,0.000000
3,Name,0.000000
4,Sex,0.000000
6,SibSp,0.000000
7,Parch,0.000000


In [14]:
train_df2 = train_df.copy()
test_df2 = test_df.copy()

In [15]:
def get_preprocessor(train_df, id_column, label):
    numerical_imputer = SimpleImputer(strategy='mean')
    categorical_imputer = SimpleImputer(strategy='most_frequent')

    categorical_variables, discrete_numerical_variables,\
    continuous_numerical_variables = data_manipulation.get_variables(train_df, [], id_column)
    discrete_numerical_variables = list(set(discrete_numerical_variables) - set([label])) 
    ind_continuous_variables = list(set(continuous_numerical_variables) - set([label]))
    numerical_variables = continuous_numerical_variables + discrete_numerical_variables

    categorical_columns_selector = selector(dtype_exclude='number')
    categorical_columns = categorical_columns_selector(train_df)
    categorical_columns = list(set(categorical_columns) - set([label]))
    #categorical_columns = ['Embarked', 'Ticket', 'Sex', 'Cabin']
    ohe2 = OneHotEncoder(handle_unknown='ignore', sparse=False)
    categorical_pipeline2 = make_pipeline(categorical_imputer, ohe2)
    
    print(f'Continuous Variables: {ind_continuous_variables}')
    print(f'Discrete Numerical Variables: {discrete_numerical_variables}')
    print(f'Categorical Variables: {categorical_columns}')
    
    preprocessor = make_column_transformer((categorical_pipeline2, categorical_columns),
                                           (numerical_imputer, ind_continuous_variables),
                                           (categorical_imputer, discrete_numerical_variables),
                                        )
    return preprocessor

In [16]:
def train_stacking_classifier(train_df, y_train, estimators, final_estimator, scoring_function):
    stacking_classifier = StackingClassifier(
            estimators=estimators,
            final_estimator=final_estimator,
        )
    
    score = cross_val_score(stacking_classifier, train_df, y_train, cv=5, scoring=scoring_function).mean()
    print(f'Score: {score}')
    return stacking_classifier, score


In [17]:
%%time
dt_param_grid = {
    'decisiontreeclassifier__max_depth':[2,3],
    'decisiontreeclassifier__min_samples_split':[10, 20, 30],
    'decisiontreeclassifier__min_samples_leaf':[10, 20, 30],
    'decisiontreeclassifier__max_features' : ['sqrt'],
    'decisiontreeclassifier__min_impurity_decrease': [i/100 for i in range(1, 3)],
    'decisiontreeclassifier__ccp_alpha': [i/10 for i in range(1, 3)]
}

rf_param_grid = {
    'randomforestclassifier__n_estimators':[100, 500, 1000],
    'randomforestclassifier__min_samples_split':[10, 20, 30, 40, 50],
    'randomforestclassifier__max_depth': [2,3,4,5],
    'randomforestclassifier__min_samples_leaf': [5, 10, 15, 20, 25, 30],
    'randomforestclassifier__min_impurity_decrease': [0.002, 0.004, 0.006, 0.008]
}

knn_param_grid = {
    'kneighborsclassifier__weights' : ['uniform'],
    'kneighborsclassifier__algorithm' : ['auto'],
    'kneighborsclassifier__n_neighbors' : [5,10,15, 20, 25, 30],
    'kneighborsclassifier__leaf_size': [10],
    'kneighborsclassifier__p': [1],
}

xgb_param_grid = {
    'xgbclassifier__n_estimators': [110, 120],
    'xgbclassifier__max_depth':[5,6],
    'xgbclassifier__max_leaves':[40, 30, 50],
    'xgbclassifier__reg_alpha': [i/100 for i in range(1,3)],
    'xgbclassifier__reg_lambda': [i/100 for i in range(1,3)],
    'xgbclassifier__colsample_bytree': [i/10 for i in range(2, 5)],
    'xgbclassifier__learning_rate': [i/100 for i in range(3,6)],
    'xgbclassifier__gamma': [i/100 for i in range(1, 3)]
}

gbr_param_grid = {
    'gradientboostingclassifier__n_estimators': [1000, 1200],
    'gradientboostingclassifier__max_depth':[4,5,6],
    'gradientboostingclassifier__max_leaf_nodes':[40, 30, 50],
    'gradientboostingclassifier__min_samples_split':[40, 30, 50],
    'gradientboostingclassifier__subsample': [i/10 for i in range(2, 5)],
    'gradientboostingclassifier__learning_rate': [i/100 for i in range(3,6)],
    'gradientboostingclassifier__max_features': [i/100 for i in range(1, 3)]
}

CPU times: total: 0 ns
Wall time: 0 ns


In [18]:
def get_headers(worksheet, row_index):
    num_rows = worksheet.max_row
    num_cols = worksheet.max_column
    headers = []
    for i in range(row_index, row_index + 1):
        for j in range(1, num_cols + 1):
            cell = worksheet.cell(row=i, column=j)
            if cell.value:
                #print(f'Cell at ({i}, {j}): {cell.value}')
                headers.append(cell.value)
    return headers

In [19]:
def save_results(workbook, worksheet_name, data, row_index, workbook_title):
    worksheet = workbook[worksheet_name]
    current_row = worksheet.max_row + 1
    headers = get_headers(worksheet, row_index)
    num_cols = len(headers)
    print(f'Num columns: {num_cols}')
    print(f'current row: {current_row}')
    cell = worksheet.cell(row=current_row - 1, column=1)
    prev_version = 0
    if cell.value != 'Version':
        current_version = cell.value + 1
        current_cell = worksheet.cell(row=current_row, column=1)
        current_cell.value = current_version
    else:
        current_cell = worksheet.cell(row=current_row, column=1)
        current_cell.value = 1
        
    for j in range(2, num_cols + 1):
        cell = worksheet.cell(row=current_row, column=j)
        cell.value = str(list(data.values())[j - 3])
        print(f'Current Row = {current_row}, current column: {j}, cell_value: {cell.value}')
    workbook.save(workbook_title)

In [20]:
def train_classifier(preprocessor, model, hyper_parameter_set, train_df, y_train, scoring_function, max_models,\
                         workbook, worksheet_name, workbook_name):
    
    final_pipe, train_score, best_params = get_pipeline_cross_val_score(preprocessor, model,\
                                            hyper_parameter_set, train_df, y_train, scoring_function, max_models)
    
    data = dict()
    feature_list = list(train_df.columns)
    feature_str = ',\n '.join(feature_list)
    
    print(f'train_score: {train_score}')
    
    #workbook_name = 'Titanic Survival Model.xlsx'
    sheet = pd.read_excel(workbook_name, sheet_name=worksheet_name).rename(columns=lambda x: x.strip())
    
    if len(sheet) > 0:
        old_version = sheet.loc[len(sheet) - 1, 'Version']
        new_version = old_version + 1
    else:
        new_version = 1
        
    public_score = 0
    sheet.loc[len(sheet)] = [new_version, feature_str, hyper_parameter_set, best_params, train_score, public_score] # row

    with pd.ExcelWriter(workbook_name,
                        mode='a', if_sheet_exists='replace') as writer:  
        sheet.to_excel(writer, sheet_name=worksheet_name, index=False)
    #row_index = 1
    #save_results(workbook, worksheet_name, data, row_index, workbook_name)
    
    return final_pipe

In [23]:
def get_attrs(obj, query=''):
    for field in dir(obj):
        if field[0:2] != '__' and field[0] != '_':
            if query.lower() in field.lower():
                print(f'Field: {field}')

In [24]:
def get_categorical_columns(train_df, label):
    categorical_columns_selector = selector(dtype_exclude='number')
    categorical_columns = categorical_columns_selector(train_df)
    categorical_columns = list(set(categorical_columns) - set([label]))
    
    return categorical_columns

def get_numerical_variables(train_df, id_column, label):
    categorical_variables, discrete_numerical_variables,\
    continuous_numerical_variables = data_manipulation.get_variables(train_df, [], id_column)
    discrete_numerical_variables = list(set(discrete_numerical_variables) - set([label])) 
    ind_continuous_variables = list(set(continuous_numerical_variables) - set([label]))
    
    return ind_continuous_variables, discrete_numerical_variables

In [25]:
def get_sex_pclass(row):
    if row['Pclass'] == 1 and row['sex_converted'] == 0:
        return 'pclass_1_female'
    elif row['Pclass'] == 2 and row['sex_converted'] == 0:
        return 'pclass_2_female'
    elif row['Pclass'] == 3 and row['sex_converted'] == 0:
        return 'pclass_3_female'
    elif row['Pclass'] == 1 and row['sex_converted'] == 1:
        return 'pclass_1_male'
    elif row['Pclass'] == 2 and row['sex_converted'] == 1:
        return 'pclass_2_male'
    else:
        return'pclass_3_male'

In [29]:
train_df['sex_converted'] = train_df['Sex'].apply(lambda x: 1 if x == 'male' else 0)

In [30]:
train_df['sex_pclass'] = train_df.apply(lambda row: get_sex_pclass(row), axis='columns')

In [39]:
train_df2 = train_df['sex_pclass'].reset_index().drop('index', axis='columns')
train_df2

,sex_pclass
0,pclass_3_male
1,pclass_1_female
2,pclass_3_female
3,pclass_1_female
4,pclass_3_male
...,...
886,pclass_2_male
887,pclass_1_female
888,pclass_3_female
889,pclass_1_male


In [41]:
preprocessor = get_preprocessor(train_df2, id_column, label)

Continuous Variables: []
Discrete Numerical Variables: []
Categorical Variables: ['sex_pclass']


In [43]:
workbook = 'useless variable'

In [44]:
%%time
dt_final_pipe = train_classifier(preprocessor, DecisionTreeClassifier, dt_param_grid, train_df2, y_train, 'accuracy', 50,\
                         workbook, 'DecisionTree', 'Titanic Survival Model.xlsx')

rf_final_pipe = train_classifier(preprocessor, RandomForestClassifier, rf_param_grid, train_df2, y_train, 'accuracy', 50,\
                         workbook, 'RandomForest', 'Titanic Survival Model.xlsx')

knn_final_pipe = train_classifier(preprocessor, KNeighborsClassifier, knn_param_grid, train_df2, y_train, 'accuracy', 50,\
                         workbook, 'KNN', 'Titanic Survival Model.xlsx')

xgb_final_pipe = train_classifier(preprocessor, XGBClassifier, xgb_param_grid, train_df2, y_train, 'accuracy', 50,\
                         workbook, 'XGBClassifier', 'Titanic Survival Model.xlsx')

gbr_final_pipe = train_classifier(preprocessor, GradientBoostingClassifier, gbr_param_grid, train_df2, y_train, 'accuracy', 50,\
                         workbook, 'GBRClassifier', 'Titanic Survival Model.xlsx')

Max models: 50
train_score: 0.6161634548992531
Max models: 50
train_score: 0.7823049400539828
Max models: 50
train_score: 0.7867365513778168
Max models: 50
train_score: 0.7867993220764548
Max models: 50
train_score: 0.773316176009039
CPU times: total: 3min 30s
Wall time: 4min 41s


In [46]:
lr_final_pipe = make_pipeline(preprocessor, LogisticRegression())

estimators = [
        ('LogisticRegression', lr_final_pipe),
         ('KNN', knn_final_pipe),
        ('RandomForest', rf_final_pipe),
        ('GradientBoostingClassifier', gbr_final_pipe),
        ('XGBClassifier', xgb_final_pipe)
]

rf_final_estimator = RandomForestClassifier()
xgb_final_estimator = XGBClassifier()

In [47]:
rf_stack, rf_stack_score = train_stacking_classifier(train_df2, y_train, estimators, rf_final_estimator, 'accuracy')

Score: 0.7766869625258929


In [48]:
vote_rf = VotingClassifier(estimators=[('LogisticRegression', lr_final_pipe),\
                                         ('KNN', knn_final_pipe),
                                         ('RF', rf_final_pipe),
                                        ('GBR', gbr_final_pipe),
                                        ('XGB', xgb_final_pipe)], voting='hard')

score = cross_val_score(vote_rf, train_df2, y_train, cv=5, scoring='accuracy').mean()

print(f'Voting Classifier with RF score: {score}')

Voting Classifier with RF score: 0.773316176009039


In [51]:
test_df['sex_converted'] = test_df['Sex'].apply(lambda x: 1 if x == 'male' else 0)
test_df['sex_pclass'] = test_df.apply(lambda row: get_sex_pclass(row), axis='columns')
test_df2 = test_df['sex_pclass'].reset_index().drop('index', axis='columns')
test_df2

,sex_pclass
0,pclass_3_male
1,pclass_3_female
2,pclass_2_male
3,pclass_3_male
4,pclass_3_female
...,...
413,pclass_3_male
414,pclass_1_female
415,pclass_3_male
416,pclass_3_male


In [52]:
predict(train_df2, y_train, test_df2, vote_rf, id_column, test_df[id_column], pred_col, label, 'vote_rf_sex_pclass.csv')